In [3]:
!pip3 install --no-deps cell-data-loader
!pip3 install tkinter


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [33]:
from cell_data_loader import CellDataloader
import torch,os,glob
from torchvision.models import resnet50 #, ResNet50_Weights
import tkinter as tk
import tkinter.filedialog

In [34]:
### CHANGE THESE LINES
wd = '/Users/mleming/Desktop/CellDataLoader_playground'
train_folder_name = os.path.join(wd,'dropbox_downloads','train')
test_folder_name = os.path.join(wd,'dropbox_downloads','test')
use_tkinter = True
root = tk.Tk()
if use_tkinter:
    print("Select your working directory to save the model")
    wd = tk.filedialog.askdirectory(parent=root,title="Please select the directory to save the model")
    print("Select the directory with the cell data")
    train_folder_name = tk.filedialog.askdirectory(title="Please select the directory with training cell data")
    test_folder_name = tk.filedialog.askdirectory(title="Please select the directory with test cell data")
print(f"Working directory to save model: {wd}")
print(f"Location of train data: {train_folder_name}")
print(f"Location of test data: {test_folder_name}")

## These strings are used to match between the files with one label and those with another
## In this case, if some folders have the word "blurry" and others have the word "clear",
## they will have labels 1 and 2. Note that CellDataLoader pushes a warning if any given
## path matches both strings (i.e., "/this/path/is/Blurry/and/Clear/img.png")

label_regex_strings = ["Blurry","Clear"]

Working directory to save model: /Users/mleming/Desktop/CellDataLoader_playground
Location of train data: /Users/mleming/Desktop/CellDataLoader_playground/dropbox_downloads/train
Location of test data: /Users/mleming/Desktop/CellDataLoader_playground/dropbox_downloads/test


In [35]:
os.makedirs(wd, exist_ok = True)

# Checkpoints
model_folder = os.path.join(wd,'checkpoints')
os.makedirs(model_folder,exist_ok=True)
model_file = os.path.join(model_folder,'torch_model.pt')

In [39]:
def train_torch(model,
                dataloader,
                model_file = None,
                epochs = 50,
                gpu_ids = None,
                verbose = True):

    if gpu_ids is not None:
        model.to(gpu_ids)

    # Train

    model.train()
    #loss_fn = torch.nn.CrossEntropyLoss()
    loss_fn = torch.nn.NLLLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
    if verbose: print("Beginning training")
    for epoch in range(epochs):
        loss_total,count = 0,0
        for image,y in dataloader_train:
            y_pred = model(image)
            y_pred = y_pred[:,:y.size()[1]]
            loss = loss_fn(y_pred, torch.argmax(y,1))
            loss_total += float(loss)
            count += 1
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if model_file is not None:
            torch.save(model,model_file)
        if verbose:
            print(
                "Epoch {epoch:d}/{epochs:d}: loss: {loss:.5f}".format(
                    epoch=epoch,epochs=epochs,loss=loss_total/count)
            )

def test_torch(model,
               dataloader,
               gpu_ids = None,
               verbose=True):
    
    model.eval()
    total_images = 0
    sum_accuracy = 0
    for image,y in dataloader:
        total_images += image.size()[0]
        y_pred = model(image)
        y_pred = y_pred[:,:y.size()[1]]
        sum_accuracy += torch.sum(torch.argmax(y_pred,axis=1) == \
            torch.argmax(y,axis=1))
    accuracy = sum_accuracy / total_images
    if verbose: print("Final accuracy: %.4f" % accuracy)


In [40]:
# Get a pretrained model from torchvision
if os.path.isfile(model_file):
    print("Loading %s" % model_file)
    model = torch.load(model_file)
else:
    # Download a pretrained resnet and edit its outputs to be compatible with cross entropy loss
    model = resnet50(pretrained=True)
    #model.fc = torch.nn.Sequential(
    #    torch.nn.Dropout(0.5),
    #    torch.nn.Linear(2048, 1000)
    #)

dataloader_train = CellDataloader(train_folder_name,label_regex=label_regex_strings,
    dtype = "torch",
    verbose = False,
    batch_size = 64,
    gpu_ids = None,
    n_channels=3)

dataloader_test = CellDataloader(test_folder_name,label_regex=label_regex_strings,
    dtype = "torch",
    verbose = False,
    batch_size = 64,
    gpu_ids = None,
    n_channels=3)

In [ ]:
train_torch(model,dataloader_train,model_file)

Beginning training
Epoch 0/50: loss: 1.03203
Epoch 1/50: loss: 0.88285
Epoch 2/50: loss: 0.80278
Epoch 3/50: loss: 0.67617
Epoch 4/50: loss: 0.70672
Epoch 5/50: loss: 0.50680
Epoch 6/50: loss: 0.44564
Epoch 7/50: loss: 0.28783
Epoch 8/50: loss: 0.39078
Epoch 9/50: loss: 0.16406
Epoch 10/50: loss: 0.19954


In [32]:
test_torch(model,dataloader_test)

tensor([[ 2.0775, -0.3837, -2.3550,  ..., -0.5430,  1.8986, -0.2228],
        [ 2.0775, -0.3837, -2.3550,  ..., -0.5430,  1.8986, -0.2228],
        [ 2.0775, -0.3837, -2.3550,  ..., -0.5430,  1.8986, -0.2228],
        ...,
        [ 2.0775, -0.3837, -2.3550,  ..., -0.5430,  1.8986, -0.2228],
        [ 2.0775, -0.3837, -2.3550,  ..., -0.5430,  1.8986, -0.2228],
        [ 2.0775, -0.3837, -2.3550,  ..., -0.5430,  1.8986, -0.2228]],
       grad_fn=<AddmmBackward0>)
tensor([[ 2.3903, -1.0563, -2.1164,  ..., -0.8507,  1.2689, -0.6499],
        [ 2.3903, -1.0563, -2.1164,  ..., -0.8507,  1.2689, -0.6499],
        [ 2.3903, -1.0563, -2.1164,  ..., -0.8507,  1.2689, -0.6499],
        ...,
        [ 2.3903, -1.0563, -2.1164,  ..., -0.8507,  1.2689, -0.6499],
        [ 2.3903, -1.0563, -2.1164,  ..., -0.8507,  1.2689, -0.6499],
        [ 2.3903, -1.0563, -2.1164,  ..., -0.8507,  1.2689, -0.6499]],
       grad_fn=<AddmmBackward0>)
tensor([[ 0.0843, -0.1752, -5.3073,  ..., -2.1159, -1.4730, -0.189